<a href="https://colab.research.google.com/github/Mithilesh1027/Mithilesh1027/blob/main/DEEP_CSAT_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project- Deep Customer Satifaction Prediction Score
Github-

In [1]:
# Imports Libraries
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
# Check TensorFlow version and confirm imports work
import tensorflow as tf

In [2]:
from google.colab import files
uploaded = files.upload()  # Upload eCommerce_Customer_support_data.csv

Saving eCommerce_Customer_support_data.csv to eCommerce_Customer_support_data.csv


In [10]:
df = pd.read_csv('eCommerce_Customer_support_data.csv')

# First, drop rows with missing 'Customer Remarks' or 'CSAT Score' safely
df = df.dropna(subset=['Customer Remarks', 'CSAT Score']).copy()

# Fill missing values without using inplace on a chained assignment
df['Item_price'] = df['Item_price'].fillna(0)
df['connected_handling_time'] = df['connected_handling_time'].fillna(0)

def map_csat(score):
    if score >= 5:
        return 2  # Satisfied
    elif score >= 3:
        return 1  # Neutral
    else:
        return 0  # Dissatisfied

# Apply mapping function safely using .loc
df['satisfaction_label'] = df['CSAT Score'].apply(map_csat)

print(df['satisfaction_label'])

print(df.info())
print(df.isnull().sum())

7        2
8        2
11       0
17       1
19       2
        ..
85887    0
85888    2
85896    2
85903    2
85904    2
Name: satisfaction_label, Length: 28742, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 28742 entries, 7 to 85904
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unique id                28742 non-null  object 
 1   channel_name             28742 non-null  object 
 2   category                 28742 non-null  object 
 3   Sub-category             28742 non-null  object 
 4   Customer Remarks         28742 non-null  object 
 5   Order_id                 22639 non-null  object 
 6   order_date_time          5824 non-null   object 
 7   Issue_reported at        28742 non-null  object 
 8   issue_responded          28742 non-null  object 
 9   Survey_response_Date     28742 non-null  object 
 10  Customer_City            5788 non-null   object 
 11  Product_catego

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28742 entries, 7 to 85904
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unique id                28742 non-null  object 
 1   channel_name             28742 non-null  object 
 2   category                 28742 non-null  object 
 3   Sub-category             28742 non-null  object 
 4   Customer Remarks         28742 non-null  object 
 5   Order_id                 22639 non-null  object 
 6   order_date_time          5824 non-null   object 
 7   Issue_reported at        28742 non-null  object 
 8   issue_responded          28742 non-null  object 
 9   Survey_response_Date     28742 non-null  object 
 10  Customer_City            5788 non-null   object 
 11  Product_category         5820 non-null   object 
 12  Item_price               28742 non-null  float64
 13  connected_handling_time  28742 non-null  float64
 14  Agent_name               28

In [12]:
df.describe()

,Item_price,connected_handling_time,CSAT Score,satisfaction_label
count,28742.000000,28742.00000,28742.000000,28742.000000
mean,1178.808434,1.20858,4.073238,1.484483
std,6274.405259,25.86896,1.560075,0.805814
min,0.000000,0.00000,1.000000,0.000000
25%,0.000000,0.00000,4.000000,1.000000
50%,0.000000,0.00000,5.000000,2.000000
75%,0.000000,0.00000,5.000000,2.000000
max,159990.000000,1277.00000,5.000000,2.000000


In [17]:
df.head()

,Unique id,channel_name,category,Sub-category,Customer Remarks,Order_id,order_date_time,Issue_reported at,issue_responded,Survey_response_Date,...,Product_category,Item_price,connected_handling_time,Agent_name,Supervisor,Manager,Tenure Bucket,Agent Shift,CSAT Score,satisfaction_label
7,372b51a5-fa19-4a31-a4b8-a21de117d75e,Inbound,Returns,Exchange / Replacement,Very good,88537e0b-5ffa-43f9-bbe2-fe57a0f4e4ae,NaN,01-08-2023 16:17,01-08-2023 16:23,01-Aug-23,...,NaN,0.0,0.0,Laura Smith,Evelyn Kimura,Jennifer Nguyen,On Job Training,Evening,5,2
8,6e4413db-4e16-42fc-ac92-2f402e3df03c,Inbound,Returns,Missing,Shopzilla app and it's all coustomer care serv...,e6be9713-13c3-493c-8a91-2137cbbfa7e6,NaN,01-08-2023 21:03,01-08-2023 21:07,01-Aug-23,...,NaN,0.0,0.0,David Smith,Nathan Patel,John Smith,>90,Split,5,2
11,4c28acf4-2ea4-4be8-b8f1-113e676fc8b7,Inbound,Order Related,Delayed,Very bad,55bbace3-eb2e-4f67-aacb-4f8ee8512f75,19-07-2023 23:50,02-08-2023 10:44,02-08-2023 11:14,02-Aug-23,...,LifeStyle,434.0,0.0,Stanley Hogan,Harper Wong,Emily Chen,>90,Split,1,0
17,c54459b3-ffb3-4ffa-a338-6fd179beb6b1,Inbound,Order Related,Installation/demo,Something,0cdaa1f1-02f2-40ec-9134-7cd0f92a6b3e,NaN,01-08-2023 10:00,01-08-2023 10:04,01-Aug-23,...,NaN,0.0,0.0,Jennifer May,Zoe Yamamoto,William Kim,On Job Training,Morning,3,1
19,6e51734e-9627-4a12-8062-877c3638627d,Inbound,Order Related,Installation/demo,All good,8a1ae638-4d8c-4a5e-bf1b-eb74f881fc3d,18-07-2023 12:20,02-08-2023 20:03,02-08-2023 20:05,02-Aug-23,...,Electronics,15990.0,0.0,David Butler,Olivia Wang,Emily Chen,31-60,Evening,5,2


In [18]:
df.tail()

,Unique id,channel_name,category,Sub-category,Customer Remarks,Order_id,order_date_time,Issue_reported at,issue_responded,Survey_response_Date,...,Product_category,Item_price,connected_handling_time,Agent_name,Supervisor,Manager,Tenure Bucket,Agent Shift,CSAT Score,satisfaction_label
85887,4b666b79-a622-4e88-b284-bbb7e5d887cf,Inbound,Returns,Reverse Pickup Enquiry,If the customer care guy and their supervisors...,330cff59-5fa6-4ffb-a5b7-64d736d375c3,NaN,31-08-2023 22:12,31-08-2023 22:20,31-Aug-23,...,NaN,0.0,0.0,Robert Lara,Brayden Wong,John Smith,31-60,Evening,1,0
85888,e88a149f-3f87-4d8f-9e2f-3d1f1a9b7525,Inbound,Returns,Reverse Pickup Enquiry,Good my problem was solved,dac0de59-0447-4448-ae08-97bdcc6fe738,NaN,31-08-2023 12:12,31-08-2023 12:13,31-Aug-23,...,NaN,0.0,0.0,Morgan Floyd,Ethan Tan,William Kim,On Job Training,Morning,5,2
85896,ceb3f6c4-6be2-4cd4-aac4-7f7416ac0519,Inbound,Payments related,Payment related Queries,Trough the Shopzilla App Refunding Policy shou...,c2b1ff0a-94b3-4a7b-811b-fc553d2c1980,NaN,31-08-2023 00:16,31-08-2023 12:23,31-Aug-23,...,NaN,0.0,0.0,Holly Huang,Charlotte Suzuki,William Kim,On Job Training,Morning,5,2
85903,44b38d3f-1523-4182-aba2-72917586647c,Inbound,Order Related,Seller Cancelled Order,Supported team customer executive good,d0e8a817-96d5-4ace-bb82-adec50398e22,NaN,31-08-2023 08:15,31-08-2023 08:17,31-Aug-23,...,NaN,0.0,0.0,Linda Foster,Noah Patel,Emily Chen,>90,Morning,5,2
85904,723bce2c-496c-4aa8-a64b-ca17004528f0,Inbound,Order Related,Order status enquiry,need to improve with proper details.,bdefe788-ccec-4eda-8ca4-51045e68db8a,NaN,31-08-2023 18:57,31-08-2023 19:02,31-Aug-23,...,NaN,0.0,0.0,Kimberly Martinez,Aiden Patel,Olivia Tan,On Job Training,Evening,5,2


In [15]:
y = to_categorical(df['satisfaction_label'], num_classes=3)

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['Customer Remarks'])
sequences = tokenizer.texts_to_sequences(df['Customer Remarks'])
text_data = pad_sequences(sequences, maxlen=100)

# Encode categorical and numerical features
le_cat = LabelEncoder()
le_subcat = LabelEncoder()
le_channel = LabelEncoder()

cat_encoded = le_cat.fit_transform(df['category'])
subcat_encoded = le_subcat.fit_transform(df['Sub-category'])
channel_encoded = le_channel.fit_transform(df['channel_name'])

scaler = MinMaxScaler()
numeric_data = scaler.fit_transform(df[['Item_price', 'connected_handling_time']])

other_features = np.column_stack((cat_encoded, subcat_encoded, channel_encoded, numeric_data))

In [14]:
X_train_text, X_test_text, X_train_other, X_test_other, y_train, y_test = train_test_split(
    text_data, other_features, y, test_size=0.2, random_state=42
)

In [13]:
text_input = Input(shape=(100,))
x = Embedding(10000, 64)(text_input)
x = LSTM(64, return_sequences=True)(x)
x = GlobalAveragePooling1D()(x)

other_input = Input(shape=(5,))
y_other = Dense(32, activation='relu')(other_input)

combined = Concatenate()([x, y_other])
z = Dense(64, activation='relu')(combined)
output = Dense(3, activation='softmax')(z)

model = Model(inputs=[text_input, other_input], outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
model.fit([X_train_text, X_train_other], y_train, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
288/288 ━━━━━━━━━━━━━━━━━━━━ 33s 100ms/step - accuracy: 0.6714 - loss: 0.9152 - val_accuracy: 0.7023 - val_loss: 0.7857
Epoch 2/10
288/288 ━━━━━━━━━━━━━━━━━━━━ 27s 95ms/step - accuracy: 0.6891 - loss: 0.8105 - val_accuracy: 0.7230 - val_loss: 0.7271
Epoch 3/10
288/288 ━━━━━━━━━━━━━━━━━━━━ 41s 95ms/step - accuracy: 0.7269 - loss: 0.7329 - val_accuracy: 0.7552 - val_loss: 0.6768
Epoch 4/10
288/288 ━━━━━━━━━━━━━━━━━━━━ 27s 95ms/step - accuracy: 0.7605 - loss: 0.6580 - val_accuracy: 0.7645 - val_loss: 0.6340
Epoch 5/10
288/288 ━━━━━━━━━━━━━━━━━━━━ 28s 98ms/step - accuracy: 0.7796 - loss: 0.6034 - val_accuracy: 0.7723 - val_loss: 0.6257
Epoch 6/10
288/288 ━━━━━━━━━━━━━━━━━━━━ 31s 106ms/step - accuracy: 0.8037 - loss: 0.5597 - val_accuracy: 0.7776 - val_loss: 0.6335
Epoch 7/10
288/288 ━━━━━━━━━━━━━━━━━━━━ 38s 96ms/step - accuracy: 0.8168 - loss: 0.5286 - val_accuracy: 0.7797 - val_loss: 0.6476
Epoch 8/10
288/288 ━━━━━━━━━━━━━━━━━━━━ 27s 95ms/step - accuracy: 0.8233 - loss: 0.5069 

In [8]:
loss, acc = model.evaluate([X_test_text, X_test_other], y_test)
print(f"Test Accuracy: {acc:.2f}")

180/180 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.7763 - loss: 0.6340
Test Accuracy: 0.77
